<a href="https://colab.research.google.com/github/Stella-Achar-Oiro/LP1-Data-Analysis-Project-2/blob/main/Stella_LP2_Week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Details
Name : Stella Achar Oiro <br>
email: stella.achar@azubiafrica.org <br>
Team: Prague <br>
Link to Github Repo - ([GitHub](https://github.com/Stella-Achar-Oiro/LP1-Data-Analysis-Project-2))
Project - Regression

#Analyzing the Relationship Between Day of the Week and Corporation Favorita's Sale
**Description:** This project is aimed at investigating any statistically significant correlations between the day of the week and sales of Corporation Favorita's main products, in order to determine whether it can be used for reliable sales forecasting.



**Null Hypothesis:** There is no statistically significant relationship between the day of the week and the sales of Corporation Favorita's main products, so it cannot be used for reliable sales forecasting. 



**Alternative Hypothesis:** There is a strong positive correlation between the day of the week and the sales of Corporation Favorita's main products, even in the presence of holidays and extreme events. Therefore, it can be used to make reliable forecasts regarding sales.



##Research Questions:


1. What is the impact of promotional campaigns on sales over the next two weeks?
2. How do customer preferences and behavior differ between different regions?
3. Is there a correlation between weather and product sales?
4. What factors influence the purchase of products during holidays or special events?


##Exploratory data analysis of the data 

In [1]:
#Importing and reviewing the data
# Data handling
import pandas as pd

# Vizualisation (Matplotlib, Plotly, Seaborn, etc. )
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
sns.set(font_scale=1.3)
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


# EDA (pandas-profiling, etc. )
# Feature Processing (Scikit-learn processing, etc. )
from sklearn import preprocessing 

# Machine Learning (Scikit-learn Estimators, Catboost, LightGBM, etc. )
from sklearn.ensemble import RandomForestRegressor 
import lightgbm as lgb
# Hyperparameters Fine-tuning (Scikit-learn hp search, cross-validation, etc. )
...

# Other packages
import os

In [2]:
from matplotlib import rcParams
#styling
plt.style.use("ggplot")
rcParams['figure.figsize'] = (12,  6)

# use sklearn to import a dataset
from sklearn.datasets import load_wine

In [3]:
# custom template for plotly
custom_template_go = {
    "layout": go.Layout(
        font={
            "size": 12,
            "color": "#666",
        },
        title={
            "font": {
                "family": "Times New Roman",
                "size": 18,
                "color": "#666",
            },
        },
        plot_bgcolor="#ffffff",
        paper_bgcolor="#ffffff",
        xaxis={
            "showgrid": False,
            "zeroline": False,
        },
        yaxis={
            "showgrid": False,
            "zeroline": False,
        },
        margin=dict(b=20,r=60,l=70,t=115),
    )
}

In [4]:
# For CSV, use pandas.read_csv
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Loading all datasets
df_holidays_events = pd.read_csv('/content/drive/MyDrive/LP2/holidays_events.csv')
df_oil = pd.read_csv('/content/drive/MyDrive/LP2/oil.csv')
df_stores = pd.read_csv('/content/drive/MyDrive/LP2/stores.csv')
df_test = pd.read_csv('/content/drive/MyDrive/LP2/test.csv')
df_train = pd.read_csv('/content/drive/MyDrive/LP2/train.csv')
df_transactions = pd.read_csv('/content/drive/MyDrive/LP2/transactions.csv')

In [6]:
#Cleaning the data and detecting possible outliers
df_holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [8]:
df_holidays_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [9]:
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [10]:
df_oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [11]:
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [12]:
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [13]:
df_test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [14]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


In [15]:
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [16]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [17]:
df_transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [18]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [19]:
#Exploring descriptive statistics such as mean, median, etc.
df_test.describe()

,id,store_nbr,onpromotion
count,2.851200e+04,28512.000000,28512.000000
mean,3.015144e+06,27.500000,6.965383
std,8.230850e+03,15.586057,20.683952
min,3.000888e+06,1.000000,0.000000
25%,3.008016e+06,14.000000,0.000000
50%,3.015144e+06,27.500000,0.000000
75%,3.022271e+06,41.000000,6.000000
max,3.029399e+06,54.000000,646.000000


In [20]:
df_test.shape

(28512, 5)

In [21]:
df_train.describe()

,id,store_nbr,sales,onpromotion
count,3.000888e+06,3.000888e+06,3.000888e+06,3.000888e+06
mean,1.500444e+06,2.750000e+01,3.577757e+02,2.602770e+00
std,8.662819e+05,1.558579e+01,1.101998e+03,1.221888e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,7.502218e+05,1.400000e+01,0.000000e+00,0.000000e+00
50%,1.500444e+06,2.750000e+01,1.100000e+01,0.000000e+00
75%,2.250665e+06,4.100000e+01,1.958473e+02,0.000000e+00
max,3.000887e+06,5.400000e+01,1.247170e+05,7.410000e+02


In [22]:
df_test.shape

(28512, 5)

In [23]:
df_test.duplicated().sum()

0

In [24]:
df_train.duplicated().sum()

0

In [25]:
print(f"Skewness: {df_train['sales'].skew()}")
print(f"Kurtosis: {df_train['sales'].kurt()}")

Skewness: 7.358757818882655
Kurtosis: 154.5620923696481


In [ ]:
#Identifying variables' contributions to overall sales


In [ ]:
#Analyzing time-series components of the data (e.g., seasonality, trends)

In [ ]:
#Conducting hypothesis testing to identify relationships between the day of the week and sales

##Identifying issues with the data

In [26]:
#Merge Tables
#Merge on the store number
df_train = pd.merge(df_train, df_stores, on="store_nbr", how="left")

df_test = pd.merge(df_test, df_stores, on="store_nbr", how="left")

df_train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13


In [27]:
#Merge data with oil data
#First Check for null's in oil data
df_oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

In [28]:
#With 43 null values in the oil data, we may have too many nulls if we merge
#Best option is to impute the null values 

#First visualise the oil data to analyse best way to impute the nulls

fig = go.Figure(go.Scatter(x=df_oil.date, y=df_oil.dcoilwtico, mode="lines", marker_color="brown"))

fig.update_layout(yaxis_title="Price",
                  yaxis_showgrid=False,
                  template=custom_template_go,
                  title={'text':"<span style='font-size:45px; color=#444; font-family:Times New Roman'>Oil Price ($)</span><br>from 2013 to 2016<b>",
                         'y':0.85, 'x':0.5, 
                         'xanchor': 'center', 'yanchor': 'top'},
                 )
fig.show()

In [29]:
#The oil data looks continious, so filling wit the last known value is a good option
df_oil.fillna(method="bfill", inplace=True)

In [30]:
fig = go.Figure(go.Scatter(x=df_oil.date, y=df_oil.dcoilwtico, mode="lines", marker_color="brown"))
fig.update_layout(yaxis_title="Price",
                  template=custom_template_go, 
                  title={'text': "<span style='font-size:45px; color=#444; font-family:Times New Roman'>Oil Price ($)</span><br>with replaced <span style='color:#6D83AA'>NaN </span>values<b>",
                         'y':0.85, 'x':0.5, 
                         'xanchor': 'center', 'yanchor': 'top'},
                 )
fig.show()

In [31]:
#Merge the train and test data with oil on date

df_train = pd.merge(df_train, df_oil, on="date", how="left")
df_test = pd.merge(df_test, df_oil, on="date", how="left")

#View the merged train data to confirm oil data is present
df_train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14


In [32]:
#Merge data with transactions data
#First Check for null's 

df_transactions.isnull().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

In [33]:
#No Nills in transactions data so we can go ahead and merge

#We will merge transactions data on both date ans store_nbr

df_train = pd.merge(df_train, df_transactions, on=["date", "store_nbr"], how="left")
df_test = pd.merge(df_test, df_transactions, on=["date", "store_nbr"], how="left")

df_train.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,93.14,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,93.14,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,93.14,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,93.14,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,93.14,NaN


In [35]:
#Do similar merge on holidays data
#First check for null values

df_holidays_events.isnull().sum()

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

In [36]:
# adding multiple holidays at the same day an unique row
df_holidays = df_holidays_events.groupby("date")[["type", "transferred", "locale", "locale_name"]].agg(lambda x: x.unique() if x.nunique() > 1 else list(x)) 


df_holidays["n_holidays"] = df_holidays.type.map(lambda x: len(x))

In [ ]:
#Reviewing data-entry conventions and practices

In [ ]:
#Checking for data rendering issues, such as null/empty values, NaNs, mismatched data types in different columns

In [ ]:
#Reviewing any visible patterns that may indicate an issue with the data, such as inconsistent ranges of values or unexpected correlations

In [ ]:
#Developing an understanding of the context behind the data to identify any potential sources of error

In [ ]:
#Conducting a more detailed analysis of individual columns or rows to further confirm any suspicions about errors in the data


##Handling identified data issues